**NOTE: To make below code working you will need your own OPEN AI API Key**

**Tutorial Code (Demo) : Data ingestion and indexing prototype using  - "Sentence Transformers and FAISS"** - Foundation of RAG framework

In [ ]:
# # # installing libraries for PDF docs
# !pip install PyPDF2
# !pip install sentence-transformers
# !pip install faiss-cpu
# !pip install --upgrade openai

**🧩 Step 1: Load / Extract (scrap) data from PDF/website**

In [ ]:
from PyPDF2 import PdfReader


def load_pdf(file_path):
    reader = PdfReader(file_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text()
    return text

In [ ]:
# extracting data from web scraping
import requests
from bs4 import BeautifulSoup
def load_webpage(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    return soup.get_text()

**🧩 Step 2: Ingesting data from PDF/website into unified a space - documents**

In [ ]:
pdf_text = load_pdf("prodigal_ai_internship_RAGTeam_details.pdf")
web_text = load_webpage("https://training.prodigalai.com/modules/ai-modules/rag-pipeline")
all_text = pdf_text + "\n" + web_text
# all_external_sources = [pdf_text, web_text]

def chunk_text(text, chunk_size=35):
    words = text.split()
    return [" ".join(words[i:i+chunk_size]) for i in range(0, len(words), chunk_size)]

documents=chunk_text(all_text)

# Optional: split into chunks
# documents = [doc.strip() for doc in all_external_sources if doc.strip()]

**Step 3: Embedding (Preprocessing for Indexing)** - Converting raw documents into semantic dense vector embeddings.

In [ ]:
from sentence_transformers import SentenceTransformer
import faiss
MODEL_NAME = 'all-MiniLM-L6-v2'
model = SentenceTransformer(MODEL_NAME)
doc_embeddings = model.encode(documents)


 **Step4. Indexing** - creating a vector index using FAISS — the core of any retrieval system.

In [ ]:
import numpy as np
dim = doc_embeddings.shape[1]
index = faiss.IndexFlatL2(dim)
index.add(np.array(doc_embeddings))

**User's Query and LLM respnse demo**

In [ ]:

from openai import OpenAI

print("\n ***Running tutorial code for - Understanding RAG model**\n :")
# Step 3: Accept a user query
query = input("Ask your query: ")
query_embedding = model.encode([query])

# Step 4: Retrieve top-k documents
top_k = 3
distances, indices = index.search(np.array(query_embedding), top_k)

THRESHOLD = 1
retrieved_chunks = []


for i, score in zip(indices[0], distances[0]):

  if score < THRESHOLD:
    retrieved_chunks.append(documents[i])


context_passage = "\n".join(retrieved_chunks)


# Step 6: Inject into LLM prompt and call OpenAI

prompt = f"""
You are a helpful assistant. Use the context below to answer the user’s question in quick short yet contentful manner and after wach sentence finishes start with new line in response.

Context:

{context_passage}

Question: {query}

Answer:"""

with open("openai_key.txt", "r") as file:
    api_key = file.read().strip()
client = OpenAI(api_key=api_key)

response = client.chat.completions.create(
    model="gpt-4-1106-preview",
    messages=[{"role": "user", "content": prompt}],
    temperature=0.4
)

print(f"\n{response.choices[0].message.content}")



 ***Running tutorial code for - Understanding RAG model**
 :
Ask your query: Tell me about team working on RAG at prodigal batch 6

The team working on the RAG project at Prodigal AI batch 6 includes coordinators Rahul Madhiwala and Trisha.
Rahul liaises with the Prodigal AI management team.
Trisha, along with Rahul, Shangna, Rajasekhar, and Kunal, are the team members.
They share a common interest in AI, specifically RAG Pipeline Architectures.
Their work on the RAG project began on May 31, 2025.
